# Analyse d'une base de données du site Ebay

Nous travaillerons sur une base de données extraite du site *eBay Kleinanzeigen* ([section allemande du site Ebay](https://www.ebay-kleinanzeigen.de/), et plus particulièrement sur les annonces de ventes de voitures d'occasion. 

Cette base de données de plus de 50 000 lignes provient du site [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data) mais a été modifiée afin d'en faciliter le nettoyage par Dataquest. 

Ce projet a plus comme objectif de nettoyer une base de données que de tirer des analyses. Je m'excuse donc d'avance pour les aficionados de belles voitures !  

In [1]:
import pandas as pd
import numpy as np

In [2]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Notre base de données est composées de 20 lignes, la plupart enregistrées au format "String". Quelques colonnes contiennent des valeurs nulles, qu'il nous faudra corriger. 

Commençons le nettoyage de données ! 

# Nettoyage de données

In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Tout d'abord nous changeons le nom de colonnes pour faciliter la compréhension.
*Les noms des colonnes étaient en effet écrits selon la convention CamelCase (cad tout attachés avec une majuscule à chaque nouveau mot), Dataquest m'a conseillé de les réécrire en respectant la convention snakecase (cad aucune majuscule et séparation des mots par "_")*

*exemple : vehiculeType devient vehicule_type* 

Il est vrai que c'est plus facile à lire et à comprendre

In [4]:
autos.columns = ["date_crawled","name","seller","offer_type","price",
                 "abtests","vehicule_type","registration_year","gearbox","power_PS",
                "model","odometer","registration_month","fuel_type","brand","unrepaired_damage",
                "ad_created","num_photos","postal_code","last_seen"]
autos.head()

,date_crawled,name,seller,offer_type,price,abtests,vehicule_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [5]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,abtests,vehicule_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-10 15:36:24,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Nous observons les anomalies suivantes : 

- Les colonnes "seller" et "offer_type" comportent une unique donnée (la même donnée apparait 49 999 fois sur une base de données de 50 000 lignes) 

- La colonne "num_photos" est étrange, il nous faut approfondir la vérification

In [6]:
autos["num_photos"].value_counts()

0    50000
Name: num_photos, dtype: int64

Il semble que la colonne "num_photos" comportent uniquement le chiffre 0. Elle nous est donc inutile, nous faisons le choix de la supprimer ainsi que supprimer la colonne "seller" et "offer_type"

In [7]:
autos = autos.drop(["num_photos","seller", "offer_type"],axis = 1)

La colonne "price" est composée de valeurs numériques mais est enregistrée au format texte. Nous supprimons les caractères "$" et "," afin de la convertir au format numérique et approfondir l'analyse. 

In [8]:
autos["price"] = autos["price"].astype(str)

autos["price"] = (autos["price"]
                          .str.replace("$","")
                          .str.replace(",","")
                          .astype(int)
                          )
autos["price"].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

Idem pour la colonne "odometer" où nous supprimons les caractères "km" et ","

In [9]:
autos["odometer"] = autos["odometer"].astype(str)

autos["odometer"]=(autos["odometer"]
                       .str.replace("km","")
                       .str.replace(",","")
                       .astype(int)
                  )

autos.rename(columns={"odometer":"odometer_km"},inplace = True)

autos["odometer_km"].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int64

# Exploration des colonnes "Price"

In [10]:
print(autos["price"].unique().shape)
print(autos["price"].describe())
autos["price"].value_counts().head()

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0       1421
500      781
1500     734
2500     643
1000     639
Name: price, dtype: int64

1421 voitures sont annoncées au prix de 0€. Représentant une infirme partie des lignes de la base de données nous pouvons les retirer de notre étude. De plus certaines voitures sont annoncées au prix d'1 000 000 € ce qui est bien trop élevé. 

Nous définissons à 351 000€ le prix maximal des annonces "serieuses" de ventes de voitures. Le prix le plus bas pouvant être un prix d'appel afin de lancer les enchères, nous conservons donc les annonces affichant un prix d'1€.

In [11]:
autos = autos[autos["price"].between(1,351000)]

autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

Il reste 48 565 annonces sur les 50 000 du début. 

# Regrouper et explorer les colonnes liées aux Dates 

In [12]:
date_added_crawler = autos[['date_crawled','ad_created','last_seen']]
print(date_added_crawler[0:5])

          date_crawled           ad_created            last_seen
0  2016-03-26 17:47:46  2016-03-26 00:00:00  2016-04-06 06:45:54
1  2016-04-04 13:38:56  2016-04-04 00:00:00  2016-04-06 14:45:08
2  2016-03-26 18:57:24  2016-03-26 00:00:00  2016-04-06 20:15:37
3  2016-03-12 16:58:10  2016-03-12 00:00:00  2016-03-15 03:16:28
4  2016-04-01 14:38:50  2016-04-01 00:00:00  2016-04-01 14:38:50


In [13]:
print(autos['date_crawled'].str[:10])

0        2016-03-26
1        2016-04-04
2        2016-03-26
3        2016-03-12
4        2016-04-01
5        2016-03-21
6        2016-03-20
7        2016-03-16
8        2016-03-22
9        2016-03-16
10       2016-03-15
11       2016-03-16
12       2016-03-31
13       2016-03-23
14       2016-03-23
15       2016-04-01
16       2016-03-16
17       2016-03-29
18       2016-03-26
19       2016-03-17
20       2016-03-05
21       2016-03-06
22       2016-03-28
23       2016-03-10
24       2016-04-03
25       2016-03-21
26       2016-04-03
28       2016-03-19
29       2016-04-02
30       2016-03-14
            ...    
49968    2016-04-01
49969    2016-03-17
49970    2016-03-21
49971    2016-03-29
49972    2016-03-26
49973    2016-03-27
49975    2016-03-27
49976    2016-03-19
49977    2016-03-31
49978    2016-04-04
49979    2016-03-20
49980    2016-03-12
49981    2016-03-15
49982    2016-03-29
49983    2016-03-06
49985    2016-04-02
49986    2016-04-04
49987    2016-03-22
49988    2016-03-28


In [14]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [15]:
(autos["last_seen"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

In [16]:
(autos["ad_created"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

In [17]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

# Gérer les erreurs présentes dans la colonne "registration_year"

Le projet datant de 2016, toutes annonces proposant un voiture ayant une date de fabrication ("registration_year") supérieure à 2016 est donc fausse. De plus, une voiture ne peut pas être trop ancienne. Un peu à la louche il est vrai (et suivant les conseils de Dataquest) je choisis comme limite la date 1900. 

Je vérifie le pourcentage d'annonces invalides c'est-à-dire ayant une registration_year non comprise entre 2016 et 1900. 

In [18]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

0.038793369710697

Seulement 4% des annonces sont incorrectes, je ne les prends donc pas en compte pour la suite de mon étude. 

In [19]:
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
Name: registration_year, dtype: float64

La plupart des annonces proposent des véhicules fabriqués entre 1995 et 2007

# Explorer le prix par marque de voiture

In [20]:
brand_counts = autos["brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > .05].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [21]:
brand_mean_prices = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)

brand_mean_prices

{'audi': 9336,
 'bmw': 8332,
 'ford': 3749,
 'mercedes_benz': 8628,
 'opel': 2975,
 'volkswagen': 5402}

En plus de retrouver majoritairement des marques allemandes, parmi le top 5 des marques de voitures que l'on retrouve le plus dans les annonces, 3 catégories se forment : 
- Les voitures ayant un prix élevé (Audi, BMW, Mercedez Benz) 
- Les voitures ayant un prix moins elevé (Ford et Opel)
- La marque Volksvagen qui se situe entre les 2 gammes de prix, ce qui explique sa popularité (marque que l'on retrouve le plus dans les annonces d'Ebay selon la base de données étudiée)

# Explorer le compteur kilomètrique, et le comparer au prix moyen par marque

In [30]:
mean_prices = pd.Series(brand_mean_prices)
pd.DataFrame(mean_prices,columns=["mean_price"])

,mean_price
audi,9336
bmw,8332
ford,3749
mercedes_benz,8628
opel,2975
volkswagen,5402


In [24]:
brand_mileage = {}

for brand in common_brands : 
    brand_only = autos[autos["brand"]==brand]
    mean_mileage = brand_only["odometer_km"].mean()
    brand_mileage[brand]=int(mean_mileage)

print(brand_mileage)

{'audi': 129157, 'opel': 129310, 'volkswagen': 128707, 'mercedes_benz': 130788, 'bmw': 132572, 'ford': 124266}


In [32]:
mileage_series = pd.Series(brand_mileage)
brand_info = pd.DataFrame(mileage_series,columns=["mean_mileage"])
brand_info["mean_price"]=mean_prices

brand_info

,mean_mileage,mean_price
audi,129157,9336
bmw,132572,8332
ford,124266,3749
mercedes_benz,130788,8628
opel,129310,2975
volkswagen,128707,5402


Ce nouveau chapitre a été très dense car j'y ai découvert 2 extensions du langage Python à savoir Numpy et Pandas. Très utiles et bien plus efficaces que les bases de codage Python que j'avais, ces extensions sont aussi très vastes. De plus elles ont chacune des particularités d'écriture (loc, iloc, à partir de 0 ou de 1), ce qui les rend plus difficiles à maitriser. Je dois encore m'entrainer dessus afin de réussir les projets utilisant Pandas et Numpy sans passer des heures sur Stack Overflow ou en utilisant la correction de Dataquest. 

Mon bilan sur ce projet est mitigé car j'ai eu beaucoup de difficulté et j'espère être mieux préparé pour les prochains chapitres (et donc projets) aprofondissant le nettoyage de données (chapitre 6 et 7). 

Après mes partiels de la semaine dernière j'ai découvert l'extension Matplotlib dans le 4ème chapitre sur la Datavisualisation et je me remets doucement dans le bain. 